# 🏛️ LIDAR考古学解析パイプライン

## 概要
このNotebookは、LIDARデータを使用した考古学的構造の検出・解析を行います。

### パイプライン
1. 🛰️ **データ取得** - .las/.laz点群データまたは合成データ
2. 🧹 **前処理** - DTM/DSM抽出、ノイズ除去
3. 🖼️ **画像生成** - Hillshade、傾斜、等高線
4. 🔀 **マルチモーダル入力** - 画像+地理情報
5. 🤖 **LLM解析** - Deepseek Vision分析
6. 🏛️ **考古学的解釈** - 構造分類と文化的解釈

## 📦 ライブラリのインストールと読み込み

In [ ]:
# 必要なライブラリのインストール
!pip install numpy matplotlib opencv-python scikit-image scipy pandas requests pillow
!pip install rasterio geopandas shapely pyproj

# 可能であれば（オプション）
# !pip install laspy pdal open3d
# !pip install openai  # OpenAI API用

In [ ]:
# ライブラリの読み込み
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import cv2
from PIL import Image, ImageDraw
import pandas as pd
import json
import os
from datetime import datetime
import base64
from io import BytesIO

# 科学計算
from scipy import ndimage
from skimage import filters, feature, measure

# 地理空間処理
try:
    import rasterio
    import geopandas as gpd
    from shapely.geometry import Point, Polygon
    GIS_AVAILABLE = True
except ImportError:
    print("GIS libraries not available - using basic functionality")
    GIS_AVAILABLE = False

# LIDAR処理（オプション）
try:
    import laspy
    LIDAR_AVAILABLE = True
except ImportError:
    print("LIDAR libraries not available - using synthetic data")
    LIDAR_AVAILABLE = False

# LLM API（オプション）
try:
    import openai
    LLM_AVAILABLE = True
except ImportError:
    print("OpenAI library not available - using mock analysis")
    LLM_AVAILABLE = False

print("✅ ライブラリの読み込み完了")

## 🛰️ Step 1: データ取得・生成

In [ ]:
def create_synthetic_lidar_data(size=(512, 512)):
    """
    合成LIDARデータの生成
    実際のLIDARファイルがない場合のテスト用
    """
    print("🔄 合成LIDARデータを生成中...")
    
    # 基本地形の生成
    x, y = np.meshgrid(np.linspace(0, 100, size[1]), np.linspace(0, 100, size[0]))
    
    # 複雑な地形パターン
    elevation = (
        100 +  # 基準標高
        30 * np.sin(x / 10) * np.cos(y / 15) +  # 大きな地形起伏
        15 * np.sin(x / 5) * np.sin(y / 8) +    # 中規模起伏
        5 * np.random.randn(*size)              # ノイズ
    )
    
    # 考古学的構造の追加
    print("🏛️ 考古学的構造を追加中...")
    
    # 1. 円形土工（Casarabe様式）
    center_y, center_x = size[0] // 3, size[1] // 3
    radius = 25
    y_coords, x_coords = np.ogrid[:size[0], :size[1]]
    circle_mask = (x_coords - center_x)**2 + (y_coords - center_y)**2 <= radius**2
    elevation[circle_mask] += 2.5  # 盛り土
    
    # 2. 線形構造（古代道路）
    elevation[size[0]//2:size[0]//2+3, :] += 1.0
    
    # 3. マウンド（儀式的構造）
    mound_y, mound_x = size[0] * 2 // 3, size[1] * 2 // 3
    mound_radius = 15
    mound_mask = (x_coords - mound_x)**2 + (y_coords - mound_y)**2 <= mound_radius**2
    mound_height = 4.0 * np.exp(-((x_coords - mound_x)**2 + (y_coords - mound_y)**2) / (mound_radius**2 / 3))
    elevation[mound_mask] += mound_height[mound_mask]
    
    # 4. 矩形構造（居住区画）
    rect_y1, rect_y2 = size[0]//4, size[0]//4 + 20
    rect_x1, rect_x2 = size[1]//2, size[1]//2 + 30
    elevation[rect_y1:rect_y2, rect_x1:rect_x2] += 1.5
    
    print(f"✅ 合成データ生成完了: {size[0]}x{size[1]}")
    return elevation

# サイト情報の設定
site_info = {
    'name': 'Amazon Test Archaeological Site',
    'region': 'Western Amazon Basin',
    'coordinates': [-8.5, -63.2],  # 緯度、経度
    'date': datetime.now().strftime('%Y-%m-%d'),
    'expected_features': ['earthworks', 'mounds', 'linear_features']
}

# データの生成または読み込み
if LIDAR_AVAILABLE:
    # 実際のLIDARファイルがある場合
    # lidar_file = "path/to/your/file.las"
    # las_data = laspy.read(lidar_file)
    # elevation = process_lidar_to_dtm(las_data)
    elevation = create_synthetic_lidar_data()
else:
    elevation = create_synthetic_lidar_data()

print(f"📊 標高データ: 最小={elevation.min():.1f}m, 最大={elevation.max():.1f}m, 平均={elevation.mean():.1f}m")

## 🧹 Step 2: 前処理とDTM/DSM抽出

In [ ]:
def preprocess_elevation_data(elevation):
    """
    標高データの前処理
    """
    print("🧹 前処理を実行中...")
    
    # 1. ノイズ除去（ガウシアンフィルタ）
    elevation_smooth = ndimage.gaussian_filter(elevation, sigma=1.0)
    
    # 2. DTM（地表面）とDSM（表面）の分離
    # 実際の処理では植生除去が必要だが、ここでは簡略化
    dtm = elevation_smooth  # 地表面
    dsm = elevation + np.random.rand(*elevation.shape) * 2  # 表面（植生含む）
    
    # 3. 地形派生データの計算
    dy, dx = np.gradient(dtm)
    slope = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))
    aspect = np.degrees(np.arctan2(-dx, dy)) % 360
    
    # 4. 曲率の計算
    dyy, dyx = np.gradient(dy)
    dxy, dxx = np.gradient(dx)
    curvature = (dxx + dyy) / (1 + dx**2 + dy**2)**(3/2)
    
    # 5. 局所起伏
    local_relief = dtm - ndimage.uniform_filter(dtm, size=20)
    
    terrain_data = {
        'dtm': dtm,
        'dsm': dsm,
        'slope': slope,
        'aspect': aspect,
        'curvature': curvature,
        'local_relief': local_relief,
        'gradient_x': dx,
        'gradient_y': dy
    }
    
    print("✅ 前処理完了")
    return terrain_data

# 前処理の実行
terrain_data = preprocess_elevation_data(elevation)

print(f"📈 傾斜: 最小={terrain_data['slope'].min():.1f}°, 最大={terrain_data['slope'].max():.1f}°")
print(f"🧭 方位: 最小={terrain_data['aspect'].min():.1f}°, 最大={terrain_data['aspect'].max():.1f}°")

## 🖼️ Step 3: 陰影起伏図と可視化画像の生成

In [ ]:
def generate_hillshade(dtm, azimuth=315, altitude=45):
    """
    陰影起伏図（Hillshade）の生成
    """
    dy, dx = np.gradient(dtm)
    slope = np.arctan(np.sqrt(dx**2 + dy**2))
    aspect = np.arctan2(-dx, dy)
    
    azimuth_rad = np.radians(azimuth)
    altitude_rad = np.radians(altitude)
    
    hillshade = np.sin(altitude_rad) * np.sin(slope) + \
               np.cos(altitude_rad) * np.cos(slope) * \
               np.cos(azimuth_rad - aspect)
    
    return ((hillshade + 1) * 127.5).astype(np.uint8)

def create_analysis_images(terrain_data, site_info):
    """
    解析用画像の生成
    """
    print("🖼️ 解析画像を生成中...")
    
    # 陰影起伏図
    hillshade = generate_hillshade(terrain_data['dtm'])
    
    # 4つの主要画像を生成
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 16))
    
    # 1. 陰影起伏図
    ax1.imshow(hillshade, cmap='gray', extent=[0, 100, 0, 100])
    ax1.set_title('Hillshade Analysis\n陰影起伏図', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Distance (km)')
    ax1.set_ylabel('Distance (km)')
    
    # 2. 標高＋等高線
    im2 = ax2.imshow(terrain_data['dtm'], cmap='terrain', extent=[0, 100, 0, 100])
    contours = ax2.contour(terrain_data['dtm'], levels=15, colors='black', alpha=0.6, linewidths=0.8)
    ax2.clabel(contours, inline=True, fontsize=8, fmt='%d m')
    ax2.set_title('Elevation + Contours\n標高＋等高線', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Distance (km)')
    ax2.set_ylabel('Distance (km)')
    plt.colorbar(im2, ax=ax2, shrink=0.8, label='Elevation (m)')
    
    # 3. 傾斜解析
    im3 = ax3.imshow(terrain_data['slope'], cmap='plasma', extent=[0, 100, 0, 100])
    ax3.set_title('Slope Analysis\n傾斜解析', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Distance (km)')
    ax3.set_ylabel('Distance (km)')
    plt.colorbar(im3, ax=ax3, shrink=0.8, label='Slope (°)')
    
    # 4. 局所起伏
    im4 = ax4.imshow(terrain_data['local_relief'], cmap='RdBu_r', extent=[0, 100, 0, 100])
    ax4.set_title('Local Relief\n局所起伏', fontsize=14, fontweight='bold')
    ax4.set_xlabel('Distance (km)')
    ax4.set_ylabel('Distance (km)')
    plt.colorbar(im4, ax=ax4, shrink=0.8, label='Relief (m)')
    
    plt.suptitle(f'LIDAR Archaeological Analysis: {site_info["name"]}\n'
                f'Coordinates: {site_info["coordinates"]}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # 画像を保存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    image_path = f'lidar_analysis_{timestamp}.png'
    plt.savefig(image_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ 解析画像保存: {image_path}")
    return {
        'hillshade': hillshade,
        'analysis_image': image_path
    }

# 画像生成
images = create_analysis_images(terrain_data, site_info)

## 🔍 Step 4: 考古学的構造の検出

In [ ]:
def detect_archaeological_structures(terrain_data):
    """
    考古学的構造の自動検出
    """
    print("🔍 考古学的構造を検出中...")
    
    dtm = terrain_data['dtm']
    local_relief = terrain_data['local_relief']
    hillshade = generate_hillshade(dtm)
    
    structures = {
        'circular_features': [],
        'linear_features': [],
        'mounds': [],
        'depressions': []
    }
    
    # 1. 円形構造の検出（Hough円変換）
    edges = cv2.Canny(hillshade, 50, 150)
    circles = cv2.HoughCircles(
        edges, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
        param1=50, param2=30, minRadius=5, maxRadius=50
    )
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            structures['circular_features'].append({
                'center': (y, x),
                'radius': int(r),
                'confidence': 0.8,
                'type': 'earthwork'
            })
    
    # 2. 線形構造の検出（Hough直線変換）
    lines = cv2.HoughLinesP(
        edges, rho=1, theta=np.pi/180, threshold=80,
        minLineLength=40, maxLineGap=8
    )
    
    if lines is not None:
        for line in lines[:10]:  # 上位10本
            x1, y1, x2, y2 = line[0]
            length = np.sqrt((x2-x1)**2 + (y2-y1)**2)
            angle = np.degrees(np.arctan2(y2-y1, x2-x1))
            
            structures['linear_features'].append({
                'start': (y1, x1),
                'end': (y2, x2),
                'length': float(length),
                'angle': float(angle),
                'confidence': 0.7,
                'type': 'road_or_canal'
            })
    
    # 3. マウンド検出（局所最大値）
    local_maxima = ndimage.maximum_filter(local_relief, size=15) == local_relief
    local_maxima_coords = np.where(local_maxima)
    
    threshold = np.std(local_relief) * 1.5
    for y, x in zip(*local_maxima_coords):
        height = local_relief[y, x]
        if height > threshold:
            structures['mounds'].append({
                'center': (y, x),
                'height': float(height),
                'confidence': min(height / threshold, 1.0),
                'type': 'ceremonial_mound'
            })
    
    # 4. 窪地検出（局所最小値）
    local_minima = ndimage.minimum_filter(local_relief, size=15) == local_relief
    local_minima_coords = np.where(local_minima)
    
    threshold_neg = -np.std(local_relief) * 1.5
    for y, x in zip(*local_minima_coords):
        depth = local_relief[y, x]
        if depth < threshold_neg:
            structures['depressions'].append({
                'center': (y, x),
                'depth': float(abs(depth)),
                'confidence': min(abs(depth) / abs(threshold_neg), 1.0),
                'type': 'quarry_or_pond'
            })
    
    # 統計情報
    total_structures = sum(len(structures[key]) for key in structures)
    structures['summary'] = {
        'total_structures': total_structures,
        'circular_count': len(structures['circular_features']),
        'linear_count': len(structures['linear_features']),
        'mound_count': len(structures['mounds']),
        'depression_count': len(structures['depressions'])
    }
    
    print(f"✅ 構造検出完了: 総計{total_structures}個")
    print(f"   - 円形構造: {len(structures['circular_features'])}個")
    print(f"   - 線形構造: {len(structures['linear_features'])}個")
    print(f"   - マウンド: {len(structures['mounds'])}個")
    print(f"   - 窪地: {len(structures['depressions'])}個")
    
    return structures

# 構造検出の実行
detected_structures = detect_archaeological_structures(terrain_data)

## 🎯 検出された構造の可視化

In [ ]:
def visualize_detected_structures(terrain_data, structures, site_info):
    """
    検出された構造の可視化
    """
    fig, ax = plt.subplots(figsize=(14, 12))
    
    # 陰影起伏図をベースに
    hillshade = generate_hillshade(terrain_data['dtm'])
    ax.imshow(hillshade, cmap='gray', alpha=0.8, extent=[0, 100, 0, 100])
    
    # 色の定義
    colors = {
        'circular_features': 'red',
        'linear_features': 'blue',
        'mounds': 'orange',
        'depressions': 'purple'
    }
    
    labels = {
        'circular_features': 'Circular Earthworks\n円形土工',
        'linear_features': 'Linear Features\n線形構造',
        'mounds': 'Mounds\nマウンド',
        'depressions': 'Depressions\n窪地'
    }
    
    # 構造をプロット
    for structure_type, color in colors.items():
        features = structures[structure_type]
        if not features:
            continue
            
        if structure_type == 'circular_features':
            for feature in features:
                center = feature['center']
                radius = feature['radius']
                # ピクセル座標を地理座標に変換
                x = (center[1] / hillshade.shape[1]) * 100
                y = 100 - (center[0] / hillshade.shape[0]) * 100
                circle = plt.Circle((x, y), radius * 100 / hillshade.shape[1], 
                                  fill=False, color=color, linewidth=2, alpha=0.8)
                ax.add_patch(circle)
                
        elif structure_type == 'linear_features':
            for feature in features:
                start = feature['start']
                end = feature['end']
                x1 = (start[1] / hillshade.shape[1]) * 100
                y1 = 100 - (start[0] / hillshade.shape[0]) * 100
                x2 = (end[1] / hillshade.shape[1]) * 100
                y2 = 100 - (end[0] / hillshade.shape[0]) * 100
                ax.plot([x1, x2], [y1, y2], color=color, linewidth=3, alpha=0.8)
                
        else:  # mounds, depressions
            x_coords, y_coords = [], []
            for feature in features:
                center = feature['center']
                x = (center[1] / hillshade.shape[1]) * 100
                y = 100 - (center[0] / hillshade.shape[0]) * 100
                x_coords.append(x)
                y_coords.append(y)
            
            if x_coords:
                ax.scatter(x_coords, y_coords, c=color, s=80, alpha=0.8, 
                          edgecolors='black', linewidth=1, label=labels[structure_type])
    
    # レジェンドとタイトル
    ax.legend(loc='upper right', bbox_to_anchor=(1, 1))
    ax.set_title(f'Archaeological Structure Detection\n考古学的構造検出\n{site_info["name"]}', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Distance (km)')
    ax.set_ylabel('Distance (km)')
    
    # 統計情報をテキストボックスで表示
    summary = structures['summary']
    stats_text = f"""検出統計:
総構造数: {summary['total_structures']}
円形: {summary['circular_count']}
線形: {summary['linear_count']}
マウンド: {summary['mound_count']}
窪地: {summary['depression_count']}"""
    
    ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    
    # 画像保存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    structure_image = f'structure_detection_{timestamp}.png'
    plt.savefig(structure_image, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ 構造検出画像保存: {structure_image}")
    return structure_image

# 構造可視化
structure_image = visualize_detected_structures(terrain_data, detected_structures, site_info)

## 🔀 Step 5: マルチモーダル入力の構築

In [ ]:
def prepare_multimodal_input(images, site_info, terrain_data, structures):
    """
    LLM解析用のマルチモーダル入力を準備
    """
    print("🔀 マルチモーダル入力を構築中...")
    
    # 地理的コンテキスト
    geographic_context = {
        'site_name': site_info['name'],
        'region': site_info['region'],
        'coordinates': site_info['coordinates'],
        'latitude': site_info['coordinates'][0],
        'longitude': site_info['coordinates'][1]
    }
    
    # 地形統計
    terrain_stats = {
        'elevation_range': {
            'min': float(terrain_data['dtm'].min()),
            'max': float(terrain_data['dtm'].max()),
            'mean': float(terrain_data['dtm'].mean()),
            'std': float(terrain_data['dtm'].std())
        },
        'slope_stats': {
            'mean': float(terrain_data['slope'].mean()),
            'max': float(terrain_data['slope'].max())
        }
    }
    
    # 構造統計
    structure_summary = structures['summary']
    
    # 分析プロンプト
    analysis_prompt = f"""
LIDAR考古学解析: {site_info['name']}

サイト情報:
- 位置: {site_info['coordinates'][0]:.3f}, {site_info['coordinates'][1]:.3f}
- 地域: {site_info['region']}
- 解析日: {site_info['date']}

地形特性:
- 標高範囲: {terrain_stats['elevation_range']['min']:.1f} - {terrain_stats['elevation_range']['max']:.1f}m
- 平均標高: {terrain_stats['elevation_range']['mean']:.1f}m
- 平均傾斜: {terrain_stats['slope_stats']['mean']:.1f}°

検出された構造:
- 総構造数: {structure_summary['total_structures']}
- 円形構造: {structure_summary['circular_count']} (土工、儀式的囲い込み)
- 線形構造: {structure_summary['linear_count']} (道路、運河、土手道)
- マウンド: {structure_summary['mound_count']} (儀式的、埋葬用構造)
- 窪地: {structure_summary['depression_count']} (採石場、貯水池)

考古学的評価をお願いします：
1. 検出されたパターンの文化的意義
2. 既知のアマゾン考古学文化との比較
3. 推定される時代と機能
4. 現地調査の推奨事項
"""
    
    multimodal_input = {
        'geographic_context': geographic_context,
        'terrain_statistics': terrain_stats,
        'structure_summary': structure_summary,
        'analysis_prompt': analysis_prompt,
        'images': images
    }
    
    print("✅ マルチモーダル入力構築完了")
    return multimodal_input

# マルチモーダル入力の準備
multimodal_input = prepare_multimodal_input(images, site_info, terrain_data, detected_structures)

# プロンプトの表示
print("📝 LLM解析プロンプト:")
print(multimodal_input['analysis_prompt'])

## 🤖 Step 6: LLM/マルチモーダルモデル解析

In [ ]:
def run_llm_analysis(multimodal_input):
    """
    LLM/マルチモーダルモデルによる解析
    """
    print("🤖 LLM解析を実行中...")
    
    if LLM_AVAILABLE:
        # 実際のLLM API呼び出し
        try:
            client = openai.OpenAI(
                api_key=os.getenv('OPENAI_API_KEY'),
                base_url=os.getenv('OPENAI_BASE_URL', 'https://openrouter.ai/api/v1')
            )
            
            response = client.chat.completions.create(
                model=os.getenv('OPENAI_MODEL', 'deepseek/deepseek-r1-0528:free'),
                messages=[
                    {
                        "role": "system",
                        "content": "あなたはアマゾン考古学の専門家です。LIDAR解析結果を基に考古学的解釈を行ってください。"
                    },
                    {
                        "role": "user",
                        "content": multimodal_input['analysis_prompt']
                    }
                ],
                temperature=0.3,
                max_tokens=1500
            )
            
            llm_analysis = response.choices[0].message.content
            
        except Exception as e:
            print(f"LLM API呼び出しエラー: {e}")
            llm_analysis = generate_mock_analysis(multimodal_input)
    else:
        # モックLLM解析
        llm_analysis = generate_mock_analysis(multimodal_input)
    
    print("✅ LLM解析完了")
    return llm_analysis

def generate_mock_analysis(multimodal_input):
    """
    モック専門家解析の生成
    """
    summary = multimodal_input['structure_summary']
    geo = multimodal_input['geographic_context']
    
    mock_analysis = f"""
🏛️ 考古学的専門家解析: {geo['site_name']}

📍 サイト評価:
位置 {geo['coordinates'][0]:.3f}, {geo['coordinates'][1]:.3f} における LIDAR 解析により、
{summary['total_structures']} の人為的構造が検出されました。

🔍 構造分析:
• 円形構造 ({summary['circular_count']}個): Casarabe文化様式の土工に類似
• 線形構造 ({summary['linear_count']}個): 古代道路網または運河システム
• マウンド ({summary['mound_count']}個): 儀式的・埋葬目的の構造
• 窪地 ({summary['depression_count']}個): 資源採取または貯水機能

🏺 文化的解釈:
検出されたパターンは以下の既知文化との類似性を示します：
- Casarabe文化 (500-1400 CE): 幾何学的土工複合体
- Marajoara文化 (400-1200 CE): 儀式的センター
- Llanos de Mojos: 耕作システムと森林島

⏰ 推定時代:
構造の特徴から、後期土器時代 (1000-1500 CE) の複合社会による
大規模な景観改変を示唆。

🎯 機能推定:
- 円形構造: 儀式的・防御的囲い込み
- 線形構造: 交通・流通ネットワーク
- マウンド: 社会階層・宗教的機能
- 複合システム: 多機能的景観利用

📋 現地調査推奨:
1. 🔬 地中レーダー調査による地下構造確認
2. 🧪 放射性炭素年代測定用サンプル採取
3. 🏺 表面セラミック分析
4. 🌱 古環境復元のための環境サンプリング
5. 👥 地域先住民コミュニティとの協議
6. 📸 フォトグラメトリー・3Dスキャン記録

⚠️ 保護措置:
検出された構造の高い密度と多様性から、即座の保護措置が必要。
地域考古学当局との連携を強く推奨。

🌟 考古学的意義:
このサイトは、アマゾン盆地の複合社会による景観改変の理解に
重要な貢献をする可能性があり、国際的な研究価値を有します。
"""
    
    return mock_analysis

# LLM解析の実行
archaeological_interpretation = run_llm_analysis(multimodal_input)

print("\n" + "="*60)
print("🤖 LLM考古学解析結果:")
print("="*60)
print(archaeological_interpretation)
print("="*60)

## 🏛️ Step 7: 考古学的評価と推奨事項

In [ ]:
def calculate_archaeological_score(structures, terrain_data):
    """
    考古学的重要度スコアの計算
    """
    summary = structures['summary']
    
    # 構造数スコア (0-0.4)
    structure_score = min(summary['total_structures'] / 50.0, 0.4)
    
    # 多様性スコア (0-0.3)
    structure_types = sum([
        1 for count in [
            summary['circular_count'],
            summary['linear_count'],
            summary['mound_count'],
            summary['depression_count']
        ] if count > 0
    ])
    diversity_score = (structure_types / 4.0) * 0.3
    
    # 地形複雑性スコア (0-0.2)
    terrain_complexity = min(terrain_data['dtm'].std() / 20.0, 0.2)
    
    # 地理的位置スコア (0-0.1) - アマゾン盆地の重要地域
    location_score = 0.1  # アマゾン地域のため高スコア
    
    total_score = structure_score + diversity_score + terrain_complexity + location_score
    return min(total_score, 1.0)

def determine_investigation_priority(score, structures):
    """
    調査優先度の決定
    """
    total_structures = structures['summary']['total_structures']
    
    if score > 0.8 or total_structures > 30:
        return "緊急 - 即座の考古学調査が必要"
    elif score > 0.6 or total_structures > 15:
        return "高 - 重要な考古学的可能性、調査推奨"
    elif score > 0.4 or total_structures > 5:
        return "中 - 適度な考古学的関心、追加解析推奨"
    else:
        return "低 - 限定的な考古学指標、監視推奨"

def generate_recommendations(score, structures, site_info):
    """
    調査推奨事項の生成
    """
    recommendations = []
    
    if score > 0.7:
        recommendations.extend([
            "即座の地域考古学当局との連携",
            "識別された構造の緊急保護措置",
            "地中レーダー調査の迅速な展開",
            "地域先住民コミュニティとの協議"
        ])
    else:
        recommendations.extend([
            "詳細な地中レーダー調査",
            "戦略的試掘プログラム",
            "高解像度ドローンマッピング"
        ])
    
    # 構造別推奨
    if structures['summary']['circular_count'] > 2:
        recommendations.append("円形構造の儀式的意義に焦点を当てた調査")
    
    if structures['summary']['linear_count'] > 3:
        recommendations.append("線形構造ネットワークの古代交通システム調査")
    
    if structures['summary']['mound_count'] > 5:
        recommendations.append("マウンドの埋葬・儀式機能に関する優先調査")
    
    # 共通推奨
    recommendations.extend([
        "絶対年代測定用放射性炭素サンプル採取",
        "表面物質存在時のセラミック分析",
        "古環境復元のための環境サンプリング",
        "フォトグラメトリー・3Dスキャンによる記録"
    ])
    
    return recommendations

# 総合評価の実行
archaeological_score = calculate_archaeological_score(detected_structures, terrain_data)
investigation_priority = determine_investigation_priority(archaeological_score, detected_structures)
recommendations = generate_recommendations(archaeological_score, detected_structures, site_info)

print("\n" + "🏛️" + "="*50)
print("総合考古学的評価")
print("="*52)
print(f"📊 考古学的スコア: {archaeological_score:.3f}/1.000")
print(f"⚠️ 調査優先度: {investigation_priority}")
print(f"🔍 検出構造総数: {detected_structures['summary']['total_structures']}")
print("\n📋 推奨事項:")
for i, rec in enumerate(recommendations, 1):
    print(f"  {i}. {rec}")
print("="*52)

## 💾 結果の保存とエクスポート

In [ ]:
def save_analysis_results(site_info, terrain_data, structures, 
                         archaeological_interpretation, archaeological_score, 
                         investigation_priority, recommendations):
    """
    解析結果の保存
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 総合結果辞書
    results = {
        'metadata': {
            'site_info': site_info,
            'analysis_timestamp': timestamp,
            'pipeline_version': '1.0'
        },
        'terrain_analysis': {
            'elevation_stats': {
                'min': float(terrain_data['dtm'].min()),
                'max': float(terrain_data['dtm'].max()),
                'mean': float(terrain_data['dtm'].mean()),
                'std': float(terrain_data['dtm'].std())
            },
            'slope_stats': {
                'mean': float(terrain_data['slope'].mean()),
                'max': float(terrain_data['slope'].max())
            }
        },
        'structure_detection': structures,
        'archaeological_assessment': {
            'overall_score': archaeological_score,
            'investigation_priority': investigation_priority,
            'llm_interpretation': archaeological_interpretation,
            'recommendations': recommendations
        }
    }
    
    # JSON保存
    json_file = f'lidar_analysis_results_{timestamp}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    # CSV保存（構造データ）
    structure_data = []
    for structure_type in ['circular_features', 'linear_features', 'mounds', 'depressions']:
        for i, structure in enumerate(structures[structure_type]):
            row = {
                'id': i,
                'type': structure_type,
                'confidence': structure.get('confidence', 0.0)
            }
            
            if 'center' in structure:
                # ピクセル座標を地理座標に概算変換
                pixel_to_geo_x = site_info['coordinates'][1] + (structure['center'][1] / 512) * 0.01
                pixel_to_geo_y = site_info['coordinates'][0] + (structure['center'][0] / 512) * 0.01
                row['longitude'] = pixel_to_geo_x
                row['latitude'] = pixel_to_geo_y
            
            # 構造固有属性
            for key in ['radius', 'height', 'depth', 'length', 'angle']:
                if key in structure:
                    row[key] = structure[key]
            
            structure_data.append(row)
    
    csv_file = f'archaeological_structures_{timestamp}.csv'
    df = pd.DataFrame(structure_data)
    df.to_csv(csv_file, index=False)
    
    # レポート保存
    report_file = f'archaeological_report_{timestamp}.md'
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(f"# LIDAR考古学解析レポート\n\n")
        f.write(f"**サイト**: {site_info['name']}\n")
        f.write(f"**座標**: {site_info['coordinates']}\n")
        f.write(f"**解析日**: {timestamp}\n\n")
        f.write(f"## 総合評価\n")
        f.write(f"- **考古学的スコア**: {archaeological_score:.3f}/1.000\n")
        f.write(f"- **調査優先度**: {investigation_priority}\n")
        f.write(f"- **検出構造数**: {structures['summary']['total_structures']}\n\n")
        f.write(f"## LLM解析\n")
        f.write(archaeological_interpretation)
        f.write(f"\n\n## 推奨事項\n")
        for i, rec in enumerate(recommendations, 1):
            f.write(f"{i}. {rec}\n")
    
    print(f"\n💾 結果保存完了:")
    print(f"   📄 JSON: {json_file}")
    print(f"   📊 CSV: {csv_file}")
    print(f"   📝 レポート: {report_file}")
    
    return {
        'json_file': json_file,
        'csv_file': csv_file,
        'report_file': report_file
    }

# 結果保存
saved_files = save_analysis_results(
    site_info, terrain_data, detected_structures,
    archaeological_interpretation, archaeological_score,
    investigation_priority, recommendations
)

## 🎯 最終結果サマリー

In [ ]:
# 最終結果の表示
print("\n" + "🎯" + "="*60)
print("LIDAR考古学解析 - 最終結果サマリー")
print("="*63)
print(f"📍 サイト: {site_info['name']}")
print(f"🌍 位置: {site_info['coordinates'][0]:.3f}, {site_info['coordinates'][1]:.3f}")
print(f"📅 解析日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n🏛️ 検出結果:")
print(f"   総構造数: {detected_structures['summary']['total_structures']}")
print(f"   - 円形構造: {detected_structures['summary']['circular_count']}個")
print(f"   - 線形構造: {detected_structures['summary']['linear_count']}個")
print(f"   - マウンド: {detected_structures['summary']['mound_count']}個")
print(f"   - 窪地: {detected_structures['summary']['depression_count']}個")
print(f"\n📊 評価:")
print(f"   考古学的スコア: {archaeological_score:.3f}/1.000")
print(f"   調査優先度: {investigation_priority}")
print(f"\n💾 生成ファイル:")
for file_type, filename in saved_files.items():
    print(f"   {file_type}: {filename}")
print("\n🔬 推奨次ステップ:")
for i, rec in enumerate(recommendations[:5], 1):
    print(f"   {i}. {rec}")
print("\n✅ 解析完了")
print("="*63)

## 📚 使用方法と拡張

### このNotebookの使用方法:
1. **実際のLIDARデータ使用**: `create_synthetic_lidar_data()` を実際の.las/.lazファイル読み込みに置き換え
2. **LLM API設定**: OpenAI/OpenRouter APIキーを環境変数に設定
3. **GIS出力**: rasterio/geopandasでShapefile等の出力機能追加

### 拡張可能な機能:
- **深層学習**: PyTorchでCNN/U-Net構造検出モデル
- **3D可視化**: Open3D/PlotlyでインタラクティブViewer
- **時系列解析**: 複数時期のLIDARデータ比較
- **WebGIS**: Folium/Leafletでインタラクティブマップ

### API設定例:
```bash
export OPENAI_API_KEY="your-openrouter-key"
export OPENAI_BASE_URL="https://openrouter.ai/api/v1"
export OPENAI_MODEL="deepseek/deepseek-r1-0528:free"
```